In [1]:
import pickle
import preprocessing
from nltk.stem import WordNetLemmatizer
from num2words import num2words

In [2]:
def apply_and(list1, list2):
    i=0
    j=0
    list3 = []
    comparisions = 0
    
    while (i<len(list1) and j<len(list2)):
        comparisions += 1
        
        if(list1[i] < list2[j]):
            i += 1
        elif(list1[i] > list2[j]):
            j += 1
        else:
            list3.append(list1[i])
            i += 1
            j += 1
    
    return list3, comparisions

def apply_or(list1, list2):
    i=0
    j=0
    list3 = []
    comparisions = 0

    
    while (i<len(list1) and j<len(list2)):
        comparisions += 1
        
        if(list1[i] < list2[j]):
            list3.append(list1[i])
            i += 1
        elif(list1[i] > list2[j]):
            list3.append(list2[j])
            j += 1
        else:
            list3.append(list1[i])
            
            i += 1
            j += 1
    
#     print(i, j)
    
    while(i<len(list1)):
        list3.append(list1[i])
        i +=1
        
    
    while(j<len(list2)):
        list3.append(list2[j])
        j += 1
        
    
        
    return list3, comparisions

def apply_not(list1):
    fullSet = [i for i in range(19998)]
    fullSet = set(fullSet)
    
    list2 = fullSet.difference(set(list1))
    return list(list2)

def check_precedence_fun(operator):
    if operator == 'not':
        return 3
    elif operator == 'and':
        return 2
    elif operator == 'or':
        return 1

def infix_postfix_fun(query):
    stack = []
    postQuery = []
    
    for word in query:
        if (word == 'or' or word == 'and' or word == 'not'):
            while len(stack) != 0 and check_precedence_fun(word) <= check_precedence_fun(stack[len(stack)-1]):
                postQuery.append(stack.pop())
            stack.append(word)
        else:
            postQuery.append(word)
    
    while len(stack) != 0:
        postQuery.append(stack.pop())
        
    return postQuery

def process_query(invertedIndex, query):
    stack = []
    comparisions = 0    
    
    for word in query:
        if word == 'and':
            list1 = stack.pop()
            list2 = stack.pop()
            list3, tempComparisions = apply_and(list1, list2)
            stack.append(list3)
            comparisions += tempComparisions
        
        elif word == 'or':
            list1 = stack.pop()
            list2 = stack.pop()
            list3, tempComparisions = apply_or(list1, list2)
            stack.append(list3)
            comparisions += tempComparisions
        
        elif word == 'not':
            list1 = stack.pop()
            stack.append(apply_not(list1))
        
        else:
            if(word in invertedIndex):
                stack.append(invertedIndex[word])
            else:
                stack.append([])
            
    return stack[len(stack)-1], comparisions

def lemmatization(file):
    lemmatizer = WordNetLemmatizer()
    new_words = []
    for word in file:
        new_words.append(lemmatizer.lemmatize(word))
    return new_words

def convert_number_to_words(file):
    new_words = []
    for word in file:
        if word.isnumeric():
            new_words.append(num2words(word))
        else:
            new_words.append(word)
    
    return new_words

def normalize_query(query):
    query = query.split(" ")
    query = lemmatization(query)
    query = convert_number_to_words(query)
    postfix = infix_postfix_fun(query)
    return postfix
    

def load_data_fun(): 
    dbfile = open('data.pkl', 'rb')
    dbfile = pickle.load(dbfile)

    docIDMapping = dbfile[0]
    docsSet = dbfile[1]
    vocab = dbfile[2]
    invertedIndex = dbfile[3]
    
    return invertedIndex

In [4]:
# query = input()
query = "123 or people or operation"
query = normalize_query(query)
invertedIndex = load_data_fun()

print(query)

retrieve_doc, comparisions = process_query(invertedIndex, query)


print("Total Comparisions: ", comparisions)
print("len of retrive data: ", len(retrieve_doc))
print("docs list: ", retrieve_doc)

['one hundred and twenty-three', 'people', 'or', 'operation', 'or']
Total Comparisions:  4904
len of retrive data:  4919
docs list:  [0, 1, 4, 5, 7, 8, 9, 11, 12, 13, 16, 17, 20, 22, 23, 24, 25, 26, 35, 40, 43, 44, 45, 51, 52, 54, 55, 60, 61, 62, 64, 66, 69, 71, 72, 75, 80, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 95, 96, 98, 99, 100, 104, 105, 110, 111, 113, 115, 116, 117, 119, 123, 126, 131, 134, 135, 136, 138, 139, 142, 144, 145, 146, 147, 150, 151, 156, 157, 161, 163, 165, 167, 173, 174, 176, 177, 181, 182, 189, 191, 195, 196, 198, 199, 200, 202, 204, 205, 207, 209, 211, 212, 217, 218, 219, 221, 222, 225, 228, 229, 232, 233, 234, 237, 238, 239, 240, 243, 244, 246, 252, 253, 259, 260, 263, 264, 265, 266, 268, 270, 271, 274, 275, 278, 280, 281, 282, 285, 293, 296, 301, 302, 304, 311, 313, 316, 318, 319, 321, 326, 328, 329, 332, 336, 338, 340, 342, 346, 348, 351, 354, 355, 357, 360, 365, 366, 368, 369, 370, 372, 373, 374, 375, 377, 378, 379, 380, 382, 384, 386, 387, 388, 389, 393, 396,